In [1]:
import numpy as np
np.random.seed(42)
np.set_printoptions(suppress=True)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import glob, time, os

import keras

Using TensorFlow backend.
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Us

In [2]:
NOTEBOOK_PATH = "C:/Users/Xetrov/Desktop/SciFair20/Code/"

In [3]:
y_df = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_grouped_1henc.csv")

In [4]:
y_df.shape

(2830743, 7)

In [5]:
y_df2 = np.argmax(y_df.to_numpy(), axis=1)

In [6]:
binarize = lambda x: 0 if x==0 else 1
binfunc = np.vectorize(binarize)

In [7]:
y_all_bin = binfunc(y_df2)

In [8]:
pd.DataFrame(y_all_bin, columns=['IsAttack']).to_csv(NOTEBOOK_PATH + "IDS2017/y_all_binary.csv", index=None)

# Apply ADASYN

In [9]:
x_scaled = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_scaled_powertransform.csv")

In [10]:
y_df = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_all_binary.csv",index_col=None)

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_valtest, y_train, y_valtest = train_test_split(x_scaled, y_df, test_size = 0.4, random_state = 42)

x_val, x_test, y_val, y_test = train_test_split(x_valtest, y_valtest, test_size = 0.5, random_state = 42)

del x_valtest 
del y_valtest

In [12]:
from imblearn.over_sampling import ADASYN

In [13]:
ada = ADASYN(random_state=42, n_jobs=-1)

In [14]:
%%time
x_train_res, y_train_res = ada.fit_resample(x_train, y_train)

C:\Users\Xetrov\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Wall time: 37min 33s


In [16]:
x_train_resdf = pd.DataFrame(x_train_res, columns=x_scaled.columns)

In [18]:
x_train_resdf.shape

(2727615, 68)

In [20]:
y_train_resdf = pd.DataFrame(y_train_res, columns=['IsAttack'])

In [22]:
y_train_resdf['IsAttack'].value_counts()

0    1364100
1    1363515
Name: IsAttack, dtype: int64

In [23]:
x_train_resdf.to_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn_binary.csv");
y_train_resdf.to_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn_binary.csv")

In [12]:
x_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn_binary.csv")

In [13]:
y_train_res = pd.read_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn_binary.csv")

In [10]:
x_train_res.drop(x_train_res.columns[0], axis=1).to_csv(NOTEBOOK_PATH + "IDS2017/x_adasyn_binary.csv", index=False)

In [11]:
y_train_res.drop(y_train_res.columns[0], axis=1).to_csv(NOTEBOOK_PATH + "IDS2017/y_adasyn_binary.csv", index=False)